In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict
from dotenv import load_dotenv
import google.generativeai as genai
import os

In [2]:
load_dotenv()

True

In [3]:
model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash", 
    api_key=os.getenv("GEMINI_API_KEY")
)

In [16]:
class BlogState(TypedDict):
    title : str
    outline : str
    content : str
    rate : int

In [5]:
def create_outline(state : BlogState) -> BlogState:
    #Fetch title
    title = state['title']
    
    #call llm gen outline
    prompt = f'Generate a detailed outline for a blog on the topic- {title}'
    outline = model.invoke(prompt)
    
    #update state
    state['outline'] = outline
    
    return state
    

In [6]:
def create_blog(state: BlogState) -> BlogState:
    title = state['title']
    outline = state['outline']
    
    prompt = f'Write a detailed blog on title - {title} using the following outline \n {outline}'
    content = model.invoke(prompt).content
    
    state['content'] = content
    
    return state

In [17]:
def evaluate(state:BlogState) -> BlogState:
    content = state['content']
    prompt = f'Based on my Content given below, rate my blog from 1-10 blog content is \n {content}'
    
    rate = model.invoke(prompt)
    
    state['rate'] = rate
    
    return state

In [18]:
graph = StateGraph(BlogState)

#Add nodes
graph.add_node('create_outline',create_outline)
graph.add_node('create_blog',create_blog)
graph.add_node('evaluate',evaluate)

#Add Edges
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline','create_blog')
graph.add_edge('create_blog','evaluate')
graph.add_edge('evaluate',END)

workflow = graph.compile()

In [19]:
#Execute
initial_state = {'title':'Rise of AI in INDIA'}
final_state = workflow.invoke(initial_state)

In [20]:
print(final_state['outline'])

content='Here\'s a detailed outline for a blog post on "The Rise of AI in India," designed to be comprehensive, engaging, and informative.\n\n---\n\n## Blog Title Options:\n*   The AI Awakening: How India is Embracing the Future of Artificial Intelligence\n*   India\'s AI Revolution: Opportunities, Challenges, and the Road Ahead\n*   From Code to Consciousness: Charting the Rise of AI in India\n*   Beyond the Hype: Understanding India\'s Unique AI Journey\n\n---\n\n## **I. Introduction (Approx. 150-200 words)**\n\n*   **A. Catchy Hook:** Start with a bold statement about the global AI revolution and then pivot to India\'s unique position.\n    *   *Example:* "Artificial Intelligence is no longer a futuristic concept; it\'s a present-day reality transforming industries and societies worldwide. And in this global AI race, India is not just a participant, but a rapidly emerging frontrunner."\n*   **B. Define AI (briefly):** A quick, accessible explanation of what AI is and its broad capab

In [21]:
print(final_state['content'])

## India's AI Revolution: Opportunities, Challenges, and the Road Ahead

Artificial Intelligence is no longer a futuristic concept; it's a present-day reality transforming industries and societies worldwide. From powering our smartphones to optimizing global supply chains, AI's influence is pervasive. And in this global AI race, India is not just a participant, but a rapidly emerging frontrunner. At its core, AI refers to the simulation of human intelligence in machines programmed to think and learn like humans, encompassing everything from machine learning and deep learning to natural language processing and computer vision.

India is uniquely positioned to become a global AI powerhouse, driven by a confluence of factors, presenting immense opportunities alongside significant challenges that require strategic navigation. This blog post will delve into the drivers behind India's AI surge, explore its transformative impact across various sectors, critically examine the opportunities and

In [22]:
print(final_state['rate'])

content='This is an exceptionally well-structured, comprehensive, and insightful blog post.\n\n**Rating: 9.5/10**\n\nHere\'s a breakdown of why, along with some minor suggestions for reaching a perfect 10:\n\n---\n\n### Strengths:\n\n1.  **Excellent Structure and Flow:** The blog is logically organized with clear headings and subheadings. It moves seamlessly from introduction to India\'s unique position, sectoral impacts, opportunities/challenges, and finally, the road ahead. This makes it very easy to read and understand.\n2.  **Comprehensive Coverage:** You\'ve touched upon almost every critical aspect of AI in India – from its foundational drivers (demographics, digital infrastructure, IT backbone) to its diverse applications across key sectors (healthcare, agriculture, finance, education, manufacturing, public services).\n3.  **Balanced Perspective:** You effectively highlight both the immense opportunities and the significant challenges (job displacement, ethics, infrastructure, f